# Methoden für den Abgleich Contouren der Teile mit einer Datenank
# Diese Idee wurde aber verworfen

In [ ]:
from typing import Dict, Any, List
from utils_camera import Camera
#from event_topics import EventTopics, create_event, EventType, ErrorEventCodes, Metadata, ProgressEventCodes, SuccessEventCodes
from datetime import datetime
from utils import ContourProcessor
#from models import ModelRegistry
import numpy as np
import cv2
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import pdist

In [ ]:
def compare_contour_to_database(self, obj):
        self.timer.start("compare_contour_to_database")
        contours = self.extract_contours_from_obj(obj)
        if not contours:
            return [], self.timer.stop("compare_contour_to_database")
        
        db_contours = self.loader.get_filtered_contours()
        diffs_and_tiles = []

        for tile_data in db_contours:
            db_contour_points = tile_data["points"]
            db_contour = ContourProcessor.format_contours_for_opencv(db_contour_points)
            for pred_contour in contours:
                cont_diff = cv2.matchShapes(pred_contour, db_contour, cv2.CONTOURS_MATCH_I1, 0)
                if cont_diff < self.similarity_threshold:
                    # Ensure that the form of the data is correct
                    tile_info = (cont_diff, tile_data["tileId"], tile_data["materialId"], tile_data["tileLabel"])
                    diffs_and_tiles.append((cont_diff, tile_data["tileId"], tile_data["materialId"], tile_data["tileLabel"]))

        identified_tiles = self.cluster_and_identify(diffs_and_tiles)
        return identified_tiles, self.timer.stop("compare_contour_to_database")

In [ ]:
def extract_contours_from_obj(self, obj):
        surfaces = obj.get("surface", [])
        if not surfaces or not surfaces[0].get("contours"):
            return []

        raw_contours = surfaces[0].get("contours", [])
        formatted_contours = [ContourProcessor.format_contours_for_opencv(contour) for contour in raw_contours]
        return formatted_contours

In [ ]:
def cluster_and_identify(self, diffs_and_tiles):
        if not diffs_and_tiles:
            return []

        if len(diffs_and_tiles) == 1:
            cont_diff, tileID, materialID, tileLabel = diffs_and_tiles[0]
            return [{"tileId": tileID, "materialId": materialID, "tileLabel": tileLabel}]

        diffs, tileIDs, materialIDs, tileLabels = zip(*diffs_and_tiles)
        Y = pdist(np.array(diffs).reshape(-1, 1), 'euclidean')
        Z = linkage(Y, 'ward')
        clusters = fcluster(Z, self.clustering_distance, criterion='distance')

        cluster_avg_diffs = {}
        for cluster_id in set(clusters):
            indices = np.where(clusters == cluster_id)[0]
            avg_diff = np.mean([diffs[i] for i in indices])
            cluster_avg_diffs[cluster_id] = avg_diff

        lowest_avg_diff_cluster = min(cluster_avg_diffs, key=cluster_avg_diffs.get)
        indices = np.where(clusters == lowest_avg_diff_cluster)[0]

        identified_tiles = [{"tileId": tileIDs[i], "materialId": materialIDs[i], "tileLabel": tileLabels[i]} for i in indices]
        return identified_tiles